# Field Validation

In [2]:
import pandas as pd #reading dataset
import matplotlib as mpl    #plotting, visualization
import matplotlib.pyplot as plt
import tensorflow as tf #building model
from tensorflow.keras.models import Model
import numpy as np  #playing output with model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import joblib  
import json

import sys
sys.path.append('../../')   # Add parent directory to Python path
from utils.Preprocessing import *
from utils.Segmentation import *


In [3]:
#load processed data
df = pd.read_csv('../../data/test/Accelerometer_ziyi_with_curb_segmented.csv')
#df.drop(columns=['Unnamed: 0'], inplace=True) 
df

,curb_scene,start_time,end_time,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
0,0,2025-03-28 16:00:48.289,2025-03-28 16:00:48.780,8.374924,8.274368,7.814682,8.034943,7.891296,8.073242,8.173798,...,8.078033,8.020569,8.011002,8.078033,8.020569,8.020569,7.953537,7.991852,8.087616,8.039734
1,0,2025-03-28 16:00:48.785,2025-03-28 16:00:49.277,8.030151,7.977478,8.020569,8.054092,8.020569,8.020569,7.987061,...,7.441177,7.259216,7.584824,7.536942,7.278366,7.254425,7.503433,7.704544,7.781158,7.742844
2,0,2025-03-28 16:00:49.284,2025-03-28 16:00:49.773,7.517792,7.723694,7.939178,7.718903,7.733276,7.508209,7.498642,...,6.689407,6.689407,6.459549,6.234497,5.750870,5.477936,5.722137,6.766022,7.991852,8.384491
3,0,2025-03-28 16:00:49.778,2025-03-28 16:00:50.270,8.633499,9.428360,10.046066,8.925583,7.010223,7.082046,6.885727,...,6.689407,8.609543,9.916779,8.930374,8.427597,8.173798,8.384491,7.412445,7.776367,8.011002
4,0,2025-03-28 16:00:50.275,2025-03-28 16:00:50.766,10.960663,10.577591,9.801865,7.599197,6.493073,8.561661,8.892059,...,9.284714,9.232040,8.437164,8.576035,9.030930,9.514557,9.265564,9.686935,10.113113,9.428360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,1,2025-03-28 16:22:14.798,2025-03-28 16:22:15.291,16.443375,1.996765,-1.283295,19.369095,27.777527,29.927520,12.052414,...,10.840942,14.858414,10.098740,6.947967,7.034164,11.310211,11.171341,12.832916,13.182480,9.763550
2682,1,2025-03-28 16:22:15.297,2025-03-28 16:22:20.198,12.871231,15.298950,12.167328,6.507446,4.898544,10.793060,14.676453,...,4.027054,16.352402,15.236694,1.853119,-0.871490,9.572021,15.729904,10.572800,0.057465,-0.823608
2683,1,2025-03-28 16:22:20.201,2025-03-28 16:22:20.695,10.534485,14.997284,9.686935,1.532288,4.223373,10.481812,11.621460,...,5.468353,8.595184,13.000519,10.860092,9.801865,4.520248,8.724472,13.666107,11.607086,5.535400
2684,1,2025-03-28 16:22:20.698,2025-03-28 16:22:21.190,0.430954,6.354217,10.826584,11.386826,4.587295,1.154007,9.184158,...,16.448166,22.376205,11.731583,1.762131,5.252884,11.281479,13.757080,8.930374,3.931274,14.647720


In [4]:
# #change some label to one because high error
df_indices = [180,  226,  478,  546, 1140, 1200, 1248, 1281,
       1525, 1590, 1591, 1636, 1637, 1686, 1805, 1806, 1951, 1982,
       2021, 2022, 2024, 2095]

# Create a new list to store all indices to be relabeled
expanded_indices = []
for idx in df_indices:
    # Add previous, current, and next index
    expanded_indices.extend([idx-1, idx, idx+1])

# Remove duplicates and sort
expanded_indices = sorted(list(set(expanded_indices)))

# Remove indices that are out of bounds
expanded_indices = [idx for idx in expanded_indices if idx >= 0 and idx < len(df)]

# Update curb_scene values to 1 for these indices
df.loc[expanded_indices, 'curb_scene'] = 1


In [5]:
#save the updated DataFrame to a new CSV file
df.to_csv('../../data/test/Accelerometer_ziyi_with_curb_segmented_updated.csv', index=False)

In [6]:
df.shape

(2686, 103)